In [1]:
import h5py
import numpy as np
import os
import random
from os.path import join
from keras import layers
from keras.models import Sequential
from keras.layers import Conv2D, Dense, GlobalAveragePooling2D, Dropout, Flatten, Activation, TimeDistributed, LSTM
from keras.utils import to_categorical
from keras_utils import set_keras_session


/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#set_keras_session()
dataset = 'UCF50'
nb_classes = int(dataset[3:])

In [3]:
def train_generator(dataset_dir, batch_size):
    
    all_files = os.listdir(dataset_dir)
    
    while True:
        images = []
        labels = []

        for _ in range(batch_size):
            
            random_filename = random.choice(all_files)
            
            with h5py.File(join(dataset_dir, random_filename), 'r') as hf:
                frames = hf['inception'][:]
                # one label
                fr_labels = hf['Y'][:]
                #random_idx = np.random.randint(frames.shape[0])
                
                images.append(frames)
                labels.append(fr_labels)
                
        sequences = np.array(images)
        labels = to_categorical(np.array(labels), nb_classes)
        
        yield sequences, labels
        
        

def valid_generator(dataset_dir, batch_size):
    
    all_files = os.listdir(dataset_dir)
    
    while True:

        idx = 0
        
        while idx < len(all_files):
            
            data_frames = []
            labels = []
            
            for _ in range(batch_size):
                
                filename = all_files[idx]

                with h5py.File(join(dataset_dir, filename), 'r') as hf:
                    frames = hf['inception'][:]
                    single_label = hf['Y'][:][0]

                    data_frames.append(frames)
                    labels.append(single_label)

                idx += 1
                if idx == len(all_files):
                    break

            sequences = np.array(data_frames)
            labels = to_categorical(np.array(labels), nb_classes)
            yield sequences, labels

In [4]:
separate_dataset_dir = join('datasets', dataset, 'separate_frames_50_h_120_w_160')
train_dir = join(separate_dataset_dir, 'train')
valid_dir = join(separate_dataset_dir, 'valid')

train_samples_count = len(os.listdir(train_dir))
valid_samples_count = len(os.listdir(valid_dir))

with h5py.File(join(train_dir, os.listdir(train_dir)[0])) as hf:
    seq_shape = hf['inception'][:].shape[:]
    print('Image input shape is', seq_shape)

Image input shape is (50, 2, 3, 2048)


In [5]:
batch_size = 4

from keras import metrics
from functools import partial

top_3_k_categorical_accuracy = partial(metrics.top_k_categorical_accuracy, k=3)
top_3_k_categorical_accuracy.__name__ = 'top_3'

model = Sequential()
model.add(TimeDistributed(GlobalAveragePooling2D(input_shape = seq_shape), input_shape = seq_shape))
model.add(LSTM(1024))
model.add(Dense(nb_classes, activation='softmax'))
print(model.summary())
model.compile(optimizer='adam', loss='categorical_crossentropy', 
              metrics=['accuracy', top_3_k_categorical_accuracy])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_1 (TimeDist (None, 50, 2048)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 1024)              12587008  
_________________________________________________________________
dense_1 (Dense)              (None, 50)                51250     
Total params: 12,638,258
Trainable params: 12,638,258
Non-trainable params: 0
_________________________________________________________________
None


In [6]:
history = model.fit_generator(train_generator(train_dir, batch_size),
                    steps_per_epoch=train_samples_count // batch_size, 
                    validation_data=valid_generator(valid_dir, batch_size),
                    validation_steps=valid_samples_count // batch_size,
                    epochs=20)

Epoch 1/20
1003/1003 [==============================] - 80s 79ms/step - loss: 3.2850 - acc: 0.2109 - top_3: 0.3674 - val_loss: 2.8801 - val_acc: 0.2989 - val_top_3: 0.4572
Epoch 2/20
1003/1003 [==============================] - 78s 78ms/step - loss: 2.0619 - acc: 0.5199 - top_3: 0.7156 - val_loss: 2.3046 - val_acc: 0.3991 - val_top_3: 0.5948
Epoch 3/20
1003/1003 [==============================] - 79s 79ms/step - loss: 1.4725 - acc: 0.6545 - top_3: 0.8308 - val_loss: 2.0759 - val_acc: 0.4549 - val_top_3: 0.6560
Epoch 4/20
1003/1003 [==============================] - 80s 80ms/step - loss: 1.0864 - acc: 0.7500 - top_3: 0.8911 - val_loss: 1.9932 - val_acc: 0.4855 - val_top_3: 0.6667
Epoch 5/20
1003/1003 [==============================] - 79s 79ms/step - loss: 0.9159 - acc: 0.7802 - top_3: 0.9220 - val_loss: 2.0645 - val_acc: 0.4595 - val_top_3: 0.6498
Epoch 6/20
1003/1003 [==============================] - 78s 78ms/step - loss: 0.7041 - acc: 0.8415 - top_3: 0.9472 - val_loss: 2.0351 - val_

KeyboardInterrupt: 